### Using BlackBoxOptim

In [1]:
include("DDEmodel.jl")
include("importData.jl")

# Import the data
pop, g2, g1, g2_0, g1_0 = get_data(joinpath("..", "data", "gem.csv"),
                                   joinpath("..", "data", "gem_pop.csv"))


# initial guess
initial_guess = log.([0.02798, 0.025502, 21.3481, 10.2881, 0.0001, 0.0001])

# trial number
j = 8

# data
data = vcat(g1[:, j]', g2[:, j]');

# do the optimization, return the estimated parameters and the objective function
min_p, _ = optimization(g1, g2, g1_0, g2_0, initial_guess, j)

# plot the long-term estimation and the original data
# PlotIt(g1_0, g2_0, j, min_p, data)

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:30
└ @ Main /home/farnazm/forwardDiff/one_state_model/model/importData.jl:30


Hereeeeeeeeeeeeeee47530.69692255779

([0.0492027, 0.0153143, 22.6754, 7.52238, 4.95807e-5, 0.025495], DiffEqObjective{getfield(DiffEqParamEstim, Symbol("##39#44")){Nothing,Bool,Int64,typeof(prob_generator),Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}},DDEProblem{Array{Float64,1},Tuple{Float64,Float64},Array{Float64,1},Tuple{},true,Array{Float64,1},DDEFunction{true,typeof(DDEmodel),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},getfield(Main, Symbol("#h#3")){Array{Float64,1},Array{Float64,1}},Nothing},MethodOfSteps{CompositeAlgorithm{Tuple{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType}},AutoSwitch{Tsit5,Rosenbrock23{0,true,DefaultLinSolve,DataType},Rational{Int64},Float64}},Nothing,Nothing,Nothing,false},L2Loss{StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}},Array{Float64,2},Nothing,Nothing,Nothing},Nothing},getfield(DiffEqParamEstim, Symbol("##43#49"))}(getfield(DiffEqParamEstim, Symbol("##39#44")){Noth

In [3]:
a = [1.0, 2.0]
sum(a)

3.0